In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# imported and named given data file as 'eyeTracker.csv'
data_df = pd.read_excel('raw.xlsx', sheet_name=['organization', 'route', 'trip_request', 'trip_summary', 'vehicle'])

Clean Trip Requests:

In [ ]:
data_df['trip_request'].head(10)

In [ ]:
print(data_df['trip_request'].shape)

# filters out nonsensical records where dropoff timestamps are before pickup timestamps...
trip_requests = data_df['trip_request'][((data_df['trip_request']['dropoff_ts'] >= data_df['trip_request']['pickup_ts']) 
                                          | (data_df['trip_request']['pickup_ts'].isna()))
                                          & (data_df['trip_request']['scheduled_ts'] >= data_df['trip_request']['requested_ts'])
                                          & (data_df['trip_request']['pickup_ts'] >= data_df['trip_request']['requested_ts'])]

print(trip_requests.shape)
proper_requests = trip_requests.drop_duplicates()
print(trip_requests.shape)

# multiple requests of same id still exist with all but one having nonsensical timestamps.
trip_requests = trip_requests.sort_values(by=['id', 'dropoff_ts', 'pickup_ts'], ascending=[True, False, False])
trip_requests = trip_requests.drop_duplicates(subset='id', keep='first')
print(trip_requests.shape)
trip_requests = trip_requests.drop(columns=['created_by', 'updated_at'])

(67082, 15)
(50972, 15)
(50972, 15)
(23561, 15)
(4027, 13)


In [ ]:
trip_requests.head(20)

Clean Route:

In [ ]:
data_df['route'].head(10)

In [ ]:
print(data_df['route'].shape)
routes = data_df['route'].drop(columns='updated_at').drop_duplicates() # remove duplicates
print(routes.shape)

(8110, 10)
(5787, 9)


In [ ]:
routes.head(10)

Merge trip_summary, routes, and trip_request:

In [67]:
trip_summary_sub = data_df['trip_summary'][['route_id', 'trip_start_lat', 'trip_start_lon', 
                                        'trip_end_lat', 'trip_end_lon', 'max_speed_mph', 
                                        'avg_speed_mph', 'trip_distance_miles']]
print(trip_summary_sub.shape)
route_summary = routes.merge(trip_summary_sub, left_on='id', right_on='route_id', suffixes=['', ''])
route_summary = route_summary.drop(columns='route_id')
print(route_summary.shape)

route_summary.to_csv('route_summary.csv')
trip_requests.to_csv('trip_requests.csv')

route_summary.head(10)

(5787, 8)
(5787, 16)


,id,driver_id,start_ts,end_ts,vehicle_id,ambulatory_riders,handicapped_riders,total_riders,status,trip_start_lat,trip_start_lon,trip_end_lat,trip_end_lon,max_speed_mph,avg_speed_mph,trip_distance_miles
0,138922,11.0,2022-02-18 20:31:50,2022-02-18 22:12:37,42,3,1,4,Completed,39.875971,-82.866503,40.118916,-83.073812,72.431,29.759932,16.76
1,138920,33.0,2022-02-18 20:51:44,2022-02-18 21:39:41,41,2,0,2,Completed,39.984041,-82.917137,39.984041,-82.917137,0.000,0.000000,0.00
2,138913,23.0,2022-02-18 13:30:02,2022-02-18 15:42:53,72,2,1,3,Completed,40.087663,-82.996966,39.971180,-82.968080,80.529,34.935207,8.04
3,138912,11.0,2022-02-18 13:16:18,2022-02-18 14:56:27,42,3,1,4,Completed,40.087793,-82.975640,39.971181,-82.968114,76.391,27.089561,8.05
4,138909,24.0,2022-02-18 11:02:26,2022-02-18 13:06:47,71,5,0,5,Completed,39.984078,-82.917179,39.971175,-82.968041,65.340,26.099655,0.89
5,138906,33.0,2022-02-18 11:40:36,2022-02-18 13:17:16,41,4,1,5,Completed,39.984055,-82.917234,39.971181,-82.968025,57.690,22.082031,0.89
6,138904,11.0,2022-02-18 11:11:57,2022-02-18 13:14:37,42,5,0,5,Completed,39.985445,-82.902549,40.087912,-82.972235,69.904,32.478186,7.07
7,138847,21.0,2022-02-23 11:16:53,2022-02-23 13:47:37,85,7,0,7,Completed,39.902781,-83.432124,40.027378,-82.939247,76.569,43.875155,8.60
8,138844,2190.0,2022-02-24 20:02:00,2022-02-24 21:42:39,46,7,0,7,Completed,40.058058,-82.959240,39.943095,-83.022471,61.538,21.883211,7.93
9,138827,45.0,2022-02-22 01:00:59,2022-02-22 01:58:07,42,0,1,1,Completed,40.146222,-83.087027,40.104194,-83.102474,45.745,22.654921,2.90


In [ ]:
# trip_requests = trip_requests.sort_values(by=['route_id', 'status', 'requested_ts'], ascending=[True, True, False])
# trip_requests = trip_requests.drop_duplicates(subset='route_id', keep='first')
# print(trip_requests.shape)
trip_reqs_sub = trip_requests[['scheduled_ts', 'pickup_ts', 'dropoff_ts', 'requested_ts', 
                               'organization_id', 'ride_type', 'estimated_miles', 'fare', 
                               'discount', 'route_id', 'special_assistance']][trip_requests['status'] == 'Completed']
route_info = route_summary.merge(trip_reqs_sub, left_on='id', right_on='route_id', how='inner', suffixes=['',''])
print(route_info.iloc[:,:13].head(10))
print(route_info.iloc[:,13:].head(10))

print(route_info.shape)
route_info = route_info.drop_duplicates()
print(route_info.shape)

       id  driver_id            start_ts              end_ts  vehicle_id  \
0  138922       11.0 2022-02-18 20:31:50 2022-02-18 22:12:37          42   
1  138922       11.0 2022-02-18 20:31:50 2022-02-18 22:12:37          42   
2  138922       11.0 2022-02-18 20:31:50 2022-02-18 22:12:37          42   
3  138922       11.0 2022-02-18 20:31:50 2022-02-18 22:12:37          42   
4  138920       33.0 2022-02-18 20:51:44 2022-02-18 21:39:41          41   
5  138913       23.0 2022-02-18 13:30:02 2022-02-18 15:42:53          72   
6  138913       23.0 2022-02-18 13:30:02 2022-02-18 15:42:53          72   
7  138913       23.0 2022-02-18 13:30:02 2022-02-18 15:42:53          72   
8  138912       11.0 2022-02-18 13:16:18 2022-02-18 14:56:27          42   
9  138912       11.0 2022-02-18 13:16:18 2022-02-18 14:56:27          42   

   ambulatory_riders  handicapped_riders  total_riders     status  \
0                  3                   1             4  Completed   
1                  3     